In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from tqdm import tqdm
import scipy.stats as scs
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import math
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from datetime import datetime, timedelta
import re

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#ds_1 = pd.read_excel('/content/drive/MyDrive/продажи.xlsx')
ds_1 = pd.read_excel('/content/drive/MyDrive/2022 к-т.xlsx')


In [ ]:
ds_1 = ds_1.drop(labels = [0], axis=0)
ds_1 = ds_1.fillna(0)
ds_1[ds_1.columns[-2]] = ds_1[ds_1.columns[-2]] * 1.57

In [ ]:
assortment = list(ds_1['Номенклатура'])
ds_1 = ds_1.drop('Номенклатура', axis=1)
ds_1 = ds_1.drop('Итого', axis=1)

In [ ]:
mean_small = ds_1[[ds_1.columns[-1], ds_1.columns[-2], ds_1.columns[-3], ds_1.columns[-4], ds_1.columns[-5], ds_1.columns[-6], ds_1.columns[-7], ds_1.columns[-8]]]

In [ ]:
mean = mean_small.mean(axis=1)

In [ ]:
mean_s = (mean).values
mean_df = pd.DataFrame(index=assortment, data=mean_s, columns=['Средние продажи по неделям'])

In [ ]:
def clear_text(text):
    s = re.sub(r'[^0123456789]', '-', text) 
    s = " ".join(s.split())
    return s
pred = []
for element in range(1, len(ds_1)+1):
  w = ds_1.loc[[element]]
  col = w.columns
  colum = []
  for row in col:
    b = row.split()
    b = clear_text(b[0])
    colum.append(b)
  date = []
  for i in colum:
    s = i[6] + i[7] + i[8] + i[9] + "-" + i[3] + i[4] + "-" + i[0] + i[1]
    date.append(s)
  val = (w.loc[element, :]).values
  df = pd.DataFrame(index=date, data=val, columns=['num_orders'])
  df.index = pd.to_datetime(df.index)

  model = SimpleExpSmoothing(df['num_orders'])
  model_fit = model.fit()
  q = model_fit.predict(len(df))
  pred.append(sum(q))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/usr/local/lib/python3.7/dis

In [ ]:
predict = pd.DataFrame(index=assortment, data=pred, columns=['Прогноз'])
predict['Прогноз новый'] = predict['Прогноз'].apply(lambda x: 0 if x < 1 else x)
predict['Прогноз новый'] = predict['Прогноз новый'].apply(round)
predict = predict.drop('Прогноз', axis=1)
predict = predict.rename_axis('Номенклатура').reset_index()

In [ ]:
predict

,Номенклатура,Прогноз новый
0,"Ассорти рыбное ""сельдь, скумбрия, горбуша"" 180...",476
1,"Ассорти рыбное ""сельдь, скумбрия, СЕМГА"" 180г....",33
2,"Ассорти рыбное ""сельдь, скумбрия, ФОРЕЛЬ"" 180г...",50
3,Берш с/м фас.1кг (1432),0
4,"Биточки ЛОСОСЕВЫЕ мороженые ПИНГВИН, 1/3кг",0
...,...,...
523,"Щука вяленая потр. ПИНГВИН, 1/3кг",3
524,"Щука ГОРЯЧЕГО КОПЧ.потр с/г. коробка, 1/3 кг",0
525,"Щука ГОРЯЧЕГО КОПЧ.потр с/г. ящик, 1/3 кг",0
526,"Щука с/м ФАСОВАННАЯ,1 кг (код 823)",1


In [ ]:
week = pd.to_datetime(date[-1])
week = week + timedelta(days=10)
week = week.strftime('%d-%m-%Y')

In [ ]:
week_predict = pd.to_datetime(date[-1])
week_predict = week_predict + timedelta(days=7)
week_predict = week_predict.strftime('%d-%m-%Y')

In [ ]:
ostatki_mv = pd.read_excel('/content/drive/MyDrive/остатки.xlsx')
general_mv = predict.merge(ostatki_mv, on='Номенклатура')
general_mv = general_mv.drop(['Ед.', 'Воровского 111А', 'Unnamed: 3', 'Unnamed: 4', 'Московская 98А', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
general_mv['Unnamed: 5'] = general_mv['Unnamed: 5'].fillna(0)
general_mv['Unnamed: 9'] = general_mv['Unnamed: 9'].fillna(0)
general_mv['Текущие остатки'] = general_mv['Unnamed: 5'] + general_mv['Unnamed: 9']
general_mv['Текущие остатки'] = general_mv['Текущие остатки'].apply(lambda x: 0 if x < 1 else x)

In [ ]:
general_mv = general_mv.drop(['Unnamed: 5', 'Unnamed: 9'], axis=1)
general_mv['Необходимый товарный запас'] = general_mv['Прогноз новый'] * 1.75
general_mv['Заказ в цех' + ' ' + 'на' + ' ' + week] = general_mv['Необходимый товарный запас'] - general_mv['Текущие остатки']
general_mv['Заказ в цех' + ' ' + 'на' + ' ' + week] = general_mv['Заказ в цех' + ' ' + 'на' + ' ' + week].apply(lambda x: 0 if x < 1 else x)
general_mv['Заказ в цех' + ' ' + 'на' + ' ' + week] = general_mv['Заказ в цех' + ' ' + 'на' + ' ' + week].apply(round)
general_mv = general_mv.rename(columns={'Прогноз новый' : 'Прогноз продаж на неделю с' + ' ' + week_predict})

In [ ]:
mean_df = mean_df.rename_axis('Номенклатура').reset_index()
general_mv = general_mv.merge(mean_df, on = 'Номенклатура', how='left')
general_mv = general_mv[[general_mv.columns[0], general_mv.columns[5], general_mv.columns[1], general_mv.columns[2], general_mv.columns[3], general_mv.columns[4]]]

In [ ]:
general_mv[general_mv.columns[1]] = general_mv[general_mv.columns[1]].apply(lambda x: 0 if x < 1 else x)
general_mv[general_mv.columns[1]] = general_mv[general_mv.columns[1]].apply(round)

In [ ]:
def group(x):
  e = x.split()
  e = ''.join(e)
  if 'вял' in e:
    return 'вяленая'
  elif 'солен' in e:
    return 'соленая'
  elif 'прян/п' in e:
    return 'соленая'
  elif 'с/с' in e:
    return 'соленая'
  elif 'С/С' in e:
    return 'соленая'
  elif 'С/СОЛ' in e:
    return 'пряного'
  elif 'ПР/П' in e:
    return 'соленая'
  elif 'ПРЯН/П' in e:
    return 'соленая'
  elif 'Прян./п' in e:
    return 'соленая'
  elif 'х/к' in e:
    return 'копченая'
  elif 'Х/К' in e:
    return 'копченая'
  elif 'Сельдьфиле' in e:
    return 'пресервы'
  elif 'СКУМБРИЯфиле' in e:
    return 'пресервы'
  elif 'Скумбрияфиле' in e:
    return 'пресервы'
  elif 'МОЗАИ' in e:
    return 'пресервы'
  elif 'салатна' in e:
    return 'пресервы'
  elif 'ОСЕНЬ' in e:
    return 'пресервы'
  elif 'аром' in e:
    return 'пресервы'
  elif 'Ассорти' in e:
    return 'пресервы'
  elif 'Салатизм/к' in e:
    return 'салаты'
  elif 'ГОРЯЧ' in e:
    return 'горячее копчение'
  elif 'ГОР/К' in e:
    return 'горячее копчение'
  elif 'ГОР/к' in e:
    return 'горячее копчение'
  elif 'Коктей' in e:
    return 'морепродукты'
  elif 'Мидии' in e:
    return 'морепродукты'
  elif 'Крев' in e:
    return 'морепродукты'
  elif 'КальмарвМ' in e:
    return 'морепродукты'
  elif 'Кальмарм' in e:
    return 'морепродукты'
  elif 'Масло' in e:
    return 'масло рыбное'
  elif 'Котл' in e:
    return 'полуфабрикаты'
  elif 'Медал' in e:
    return 'полуфабрикаты'
  else:
    return 'прочее'

In [ ]:
general_mv['Группа'] = general_mv['Номенклатура'].apply(group)


In [ ]:
general_mv

,Номенклатура,Средние продажи по неделям,Прогноз продаж на неделю с 30-05-2022,Текущие остатки,Необходимый товарный запас,Заказ в цех на 02-06-2022,Группа
0,"Ассорти рыбное ""сельдь, скумбрия, горбуша"" 180...",491,476,451.000,833.00,382,пресервы
1,"Ассорти рыбное ""сельдь, скумбрия, СЕМГА"" 180г....",20,33,58.000,57.75,0,пресервы
2,"Ассорти рыбное ""сельдь, скумбрия, ФОРЕЛЬ"" 180г...",30,50,9.000,87.50,78,пресервы
3,"Брюшки лосося соленые (НЕВОЗВРАТНАЯ ТАРА), 1/2кг",0,0,60.000,0.00,0,соленая
4,"Брюшки лосося соленые ВЕСОВЫЕ,1/3кг ТОЛЬКО НА ...",6,10,60.000,17.50,0,соленая
...,...,...,...,...,...,...,...
271,Хамса с/м черноморская фасованная (код944),35,34,29.130,59.50,30,прочее
272,"Чехонь вяленая Пингвин, 1/3кг",15,85,7.400,148.75,141,вяленая
273,"Чехонь х/к ,1/3кг",12,12,29.400,21.00,0,копченая
274,"Щука вяленая потр. ПИНГВИН, 1/3кг",18,3,3.000,5.25,2,вяленая


In [ ]:
old_order = pd.read_excel('/content/drive/MyDrive/Заявки/Заявка  с 26 05 2022 по 01 06 2022.xlsx')
old_order = old_order.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 	'Unnamed: 4', 'Unnamed: 5', 	'Unnamed: 6', 	'Unnamed: 7', 	'Unnamed: 8', 	'Unnamed: 9', 	'Unnamed: 10', 'Unnamed: 12'], axis=1)

In [ ]:
old_order = old_order.rename(columns={old_order.columns[0] : 'Номенклатура'})

In [ ]:
e = ['соленая', 'копченая', 'пресервы']

In [ ]:
general_mv_small = general_mv.query('Группа == @e')

In [ ]:
p = []
for element in general_mv_small['Номенклатура']:
  q = element.split()
  q = ' '.join(q)
  p.append(q)


In [ ]:
t = []
for element in old_order['Номенклатура']:
  try:
    a = element.split()
    a = ' '.join(a)
    t.append(a)
  except:
    t.append(element)

In [ ]:
old_order['Номенклатура'] = t

In [ ]:
general_mv_small['Номенклатура'] = p

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
week_old = pd.to_datetime(date[-1])
week_old = week_old + timedelta(days=3)
week_old = week_old.strftime('%d-%m-%Y')

In [ ]:
general_mv_small_all = general_mv_small.merge(old_order, on='Номенклатура', how='left')
general_mv_small_all = general_mv_small_all.rename(columns={general_mv_small_all.columns[7] : 'Заказ с' + ' ' + week_old})

In [ ]:
general_mv_small_all = general_mv_small_all[[general_mv_small_all.columns[0], general_mv_small_all.columns[1], general_mv_small_all.columns[2], general_mv_small_all.columns[7], general_mv_small_all.columns[3], general_mv_small_all.columns[4], general_mv_small_all.columns[5], general_mv_small_all.columns[6]]]

In [ ]:
general_mv_small_all = general_mv_small_all.fillna(0)
general_mv_small_all

,Номенклатура,Средние продажи по неделям,Прогноз продаж на неделю с 30-05-2022,Заказ с 26-05-2022,Текущие остатки,Необходимый товарный запас,Заказ в цех на 02-06-2022,Группа
0,"Ассорти рыбное ""сельдь, скумбрия, горбуша"" 180...",491,476,0,451.0,833.00,382,пресервы
1,"Ассорти рыбное ""сельдь, скумбрия, СЕМГА"" 180г....",20,33,0,58.0,57.75,0,пресервы
2,"Ассорти рыбное ""сельдь, скумбрия, ФОРЕЛЬ"" 180г...",30,50,0,9.0,87.50,78,пресервы
3,"Брюшки лосося соленые (НЕВОЗВРАТНАЯ ТАРА), 1/2кг",0,0,120,60.0,0.00,0,соленая
4,"Брюшки лосося соленые ВЕСОВЫЕ,1/3кг ТОЛЬКО НА ...",6,10,60,60.0,17.50,0,соленая
...,...,...,...,...,...,...,...,...
187,"Форель х/к П/Н, 1/3кг",18,19,0,0.0,33.25,33,копченая
188,"Форель х/к СПИНКА весовая , 1/4кг",39,55,40,27.4,96.25,69,копченая
189,"Форель х/к ТЕША весовая ЯЩИК, 1/4кг",9,6,0,8.2,10.50,2,копченая
190,"Форель х/к филе на подлож.в/у 200г (1*2шт), 1/...",17,4,36,1.0,7.00,6,копченая


In [ ]:
general_mv_small_all[general_mv_small_all.columns[6]] = general_mv_small_all[general_mv_small_all.columns[5]] - ((general_mv_small_all[general_mv_small_all.columns[3]] + general_mv_small_all[general_mv_small_all.columns[4]]) - general_mv_small_all[general_mv_small_all.columns[2]])

In [ ]:
general_mv_small_all[general_mv_small_all.columns[6]] = general_mv_small_all[general_mv_small_all.columns[6]].apply(lambda x: 0 if x < 1 else x)
general_mv_small_all[general_mv_small_all.columns[6]] = general_mv_small_all[general_mv_small_all.columns[6]].apply(round)

In [ ]:
general_mv_small_all.to_excel('./all.xlsx')

In [ ]:
general_mv.to_excel('./заказ с учетом остатков.xlsx')

In [ ]:
general_mv

,Номенклатура,Средние продажи по неделям,Прогноз продаж на неделю с 30-05-2022,Текущие остатки,Необходимый товарный запас,Заказ в цех на 02-06-2022,Группа
0,"Ассорти рыбное ""сельдь, скумбрия, горбуша"" 180...",491,476,451.000,833.00,382,пресервы
1,"Ассорти рыбное ""сельдь, скумбрия, СЕМГА"" 180г....",20,33,58.000,57.75,0,пресервы
2,"Ассорти рыбное ""сельдь, скумбрия, ФОРЕЛЬ"" 180г...",30,50,9.000,87.50,78,пресервы
3,"Брюшки лосося соленые (НЕВОЗВРАТНАЯ ТАРА), 1/2кг",0,0,60.000,0.00,0,соленая
4,"Брюшки лосося соленые ВЕСОВЫЕ,1/3кг ТОЛЬКО НА ...",6,10,60.000,17.50,0,соленая
...,...,...,...,...,...,...,...
271,Хамса с/м черноморская фасованная (код944),35,34,29.130,59.50,30,прочее
272,"Чехонь вяленая Пингвин, 1/3кг",15,85,7.400,148.75,141,вяленая
273,"Чехонь х/к ,1/3кг",12,12,29.400,21.00,0,копченая
274,"Щука вяленая потр. ПИНГВИН, 1/3кг",18,3,3.000,5.25,2,вяленая
